<a href="https://colab.research.google.com/github/nithinkrishna2019/100-image-classification/blob/master/INfected_vs_uninfected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
pwd

'/content'

In [4]:
os.chdir("/content/drive/My Drive/data")

In [5]:
PATH=os.getcwd()

In [6]:
DATA_PATH=os.path.join(PATH,'/content/drive/My Drive/data')
data_dir_list=os.listdir(DATA_PATH)

In [7]:
print(data_dir_list)

['infected', 'Uninfected']


In [8]:
img_rows=224
img_cols=224
num_channels=3

img_data_list=[]
classes_names_list=[]

In [9]:
num_epoch=10
batch_size=64

In [10]:
import cv2

In [11]:
for dataset in data_dir_list:
    classes_names_list.append(dataset) 
    print ('Loading images from {} folder\n'.format(dataset)) 
    img_list=os.listdir(DATA_PATH+'/'+ dataset)
    for img in img_list:
        input_img=cv2.imread(DATA_PATH + '/'+ dataset + '/'+ img )
        input_img_resize=cv2.resize(input_img,(img_rows, img_cols))
        img_data_list.append(input_img_resize)

Loading images from infected folder

Loading images from Uninfected folder



In [12]:
data_dir_list

['infected', 'Uninfected']

In [13]:
len(input_img)

133

In [14]:
len(input_img_resize)

224

In [15]:
type(img_data_list)

list

In [16]:
num_classes=len(classes_names_list)
print(num_classes)

2


In [17]:
img_data=np.array(img_data_list)
img_data=img_data.astype('float32')

In [18]:
img_data /= 255

In [19]:
print(img_data.shape)

(2999, 224, 224, 3)


In [20]:
num_of_samples=img_data.shape[0]
input_shape=img_data[0].shape

In [21]:
print(num_of_samples)

2999


In [22]:
print(input_shape)


(224, 224, 3)


In [23]:
classes=np.ones((num_of_samples,),dtype='int64')
classes[0:1501]=1 #infected
classes[1501:]=0   #uninfected

In [24]:

from keras.utils import to_categorical

classes = to_categorical(classes, num_classes)

Using TensorFlow backend.


In [25]:
from sklearn.utils import shuffle

X, Y = shuffle(img_data, classes, random_state=2)

In [26]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [27]:
y_test.shape

(600, 2)

In [28]:
y_train.shape

(2399, 2)

In [42]:
from keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(
    rotation_range=20, #rotation ranges b/w 0 to 20
    shear_range=0.5, 
    zoom_range=0.4, 
    vertical_flip=True, 
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [43]:
test_data_gen = ImageDataGenerator()

In [44]:

ftrain_generator = train_data_gen.flow(
        X_train,
        y_train,
        batch_size=batch_size, 
        shuffle=True)

In [45]:
ftest_generator = test_data_gen.flow(
        X_test,
        y_test,
        batch_size=batch_size, 
        shuffle=True)

In [46]:
from keras.layers import Input, Dense

In [47]:
image_input = Input(shape=(img_rows, img_cols, num_channels))

In [48]:
from keras.applications.vgg16 import VGG16

model = VGG16(input_tensor=image_input, include_top=True, weights='imagenet')

In [49]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [50]:
last_layer = model.get_layer('fc2').output
out = Dense(num_classes, activation='softmax', name='output')(last_layer)

In [51]:
from keras.models import Model

custom_vgg_model = Model(image_input, out)
custom_vgg_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [52]:
for layer in custom_vgg_model.layers[:-1]:
    layer.trainable = False

In [56]:
custom_vgg_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [57]:

custom_vgg_model.fit_generator(ftrain_generator, epochs =num_epoch , validation_data=ftest_generator,validation_steps=25,steps_per_epoch=X_train.shape[0]/batch_size)

Epoch 1/10
38/37 [==============================] - 34s 905ms/step - loss: 0.5851 - accuracy: 0.7011 - val_loss: 0.3473 - val_accuracy: 0.8066
Epoch 2/10
38/37 [==============================] - 29s 769ms/step - loss: 0.5069 - accuracy: 0.7649 - val_loss: 0.4088 - val_accuracy: 0.7493
Epoch 3/10
38/37 [==============================] - 29s 768ms/step - loss: 0.4883 - accuracy: 0.7774 - val_loss: 0.4445 - val_accuracy: 0.8316
Epoch 4/10
38/37 [==============================] - 29s 766ms/step - loss: 0.4901 - accuracy: 0.7737 - val_loss: 0.4248 - val_accuracy: 0.8162
Epoch 5/10
38/37 [==============================] - 29s 774ms/step - loss: 0.4884 - accuracy: 0.7724 - val_loss: 0.4036 - val_accuracy: 0.8316
Epoch 6/10
38/37 [==============================] - 29s 766ms/step - loss: 0.4765 - accuracy: 0.7820 - val_loss: 0.3342 - val_accuracy: 0.7696
Epoch 7/10
38/37 [==============================] - 29s 772ms/step - loss: 0.4715 - accuracy: 0.7787 - val_loss: 0.3426 - val_accuracy: 0.8691